 this notebook used microsoft academic graph to estimate the true positives of two authors being the same give matching coauthors

In [30]:
import sys
sys.path.append('C:/Users/leo valberg/dev/Microsoft Academic') # path for akapi - academic knowledge api
import akapi as q

import random
import pandas as pd
import time

from pprint import pprint

In [14]:
df = pd.read_csv('parsed_names_3.csv') # need a dataframe with col 'norm' of normalised names 


,author_string_id,error,forenames,forenames_array,format,initials,initials_array,journal_id,last_names,last_names_array,norm,paper_id,raw,first_initial,last_name
0,623690,,,[''],initials_names,a,['a'],528.0,alvarez d'amelio,"['alvarez', ' ""damelio""']",a alvarez d'amelio,230306.0,A Alvarez D'Amelio,a,"""damelio"""
1,72191,,al,['al'],names,a,['a'],82.0,al ameen a,"['al', ' ameen', ' a']",al ameen a,17247.0,Al Ameen A,a,a
2,375500,,ahmad ramazani s,"['ahmad', ' ramazani', ' s']",names,a r s,"['a', ' r', ' s']",80.0,s a,"['s', ' a']",ahmad ramazani s a,122742.0,Ahmad Ramazani S. A.,a,a
3,314008,,ahmad ramazani s,"['ahmad', ' ramazani', ' s']",names,a r s,"['a', ' r', ' s']",149.0,s a,"['s', ' a']",ahmad ramazani s a,100710.0,Ahmad Ramazani S.A.,a,a
4,149116,,annegret van,"['annegret', ' van']",names,a v,"['a', ' v']",415.0,van der aa,"['van', ' der', ' aa']",annegret van der aa,43439.0,Annegret Van der Aa,a,aa


In [95]:
def attempt_request(f, params):
    
    max_attempts = 10
    
    attempts = 0
    while attempts < max_attempts:
        res = f(params)
        if 'entities' in res:
            return res
        time.sleep((2 ** attempts) + random.random())
        attempts = attempts + 1
        
    print('max attempts')
    return False

In [276]:
def get_source_id(df):
    
        index = random.randint(0, len(df))
        name  = df.iloc[index].norm
        #print(name)
        
        params = {
            # Request parameters
            'expr': "AuN=='{}'".format(name),
            'model': 'latest',
            'count': '10',
            'offset': '0',
            #'orderby': '{string}',
            'attributes': 'Id',
        }        
        res = attempt_request(q.evaluate, params)
        
        #print(res['entities'])
        
        if res == False: return False, False, False           

        id_count = len(res['entities'])

        if id_count == 0: return False, False, False
        
        source_id = res['entities'][0]['Id']

        return source_id, name, id_count

In [277]:
def get_paper_coauthors(source_id):
    
    # get the papers for that author id
    params = {
        # Request parameters
        'expr': "Composite(AA.AuId={})".format(source_id),
        #'expr': "Id==1976088248",
        'model': 'latest',
        'count': '1000',
        'offset': '0',
        #'orderby': '{string}',
        'attributes': 'AA.AuId',
    }
    return attempt_request(q.evaluate, params)

def get_coauthor_ids(obj, source_id):
    co_id = []
    for co in obj['AA']:
        if co['AuId'] != source_id:
            co_id.append(co['AuId'])
            
    return co_id


# get the probability that will have one or more coauthors - given it is the same author P(X|A)

In [264]:
# create a new empty dataframe
tp = pd.DataFrame() # probability of coauthors

In [293]:
def get_tp(df, n):
    
    # create empty true positives dataframe
    tp = pd.DataFrame()

    for i in range(n):
        
        source_id, name, id_count = get_source_id(df)            
        if source_id == False: continue
            
        res = get_paper_coauthors(source_id)
        if res == False: continue

        # if there are more than one paper
        papers = len(res['entities']) - 1

        co_count = 0 # count of papers with one or more matches
        dmatch_count = {} # dictionary with distribution of counts

        if papers > 1:

            # select the first paper and create an array of coauthor ids
            source = get_coauthor_ids(res['entities'][0], source_id)

            # loop through the remaning papers 
            # and count the number of matching coauthors
            for i in range(1, len(res['entities'])):

                flag = False
                match_count = 0

                for co in res['entities'][i]['AA']:
                    if co['AuId'] in source:
                        flag = True
                        match_count += 1

                if flag: co_count += 1
                if match_count in dmatch_count:
                    dmatch_count[match_count] += 1
                else:
                    dmatch_count[match_count] = 1

        row = {
            'name': name, 
            'akid': source_id,
            'id_count': id_count, 
            'papers': papers, 
            'co_count': co_count,
            'dmatch': dmatch_count
        }

        for i in range(3):
            if i in dmatch_count:
                row['matched_' + str(i)] = dmatch_count[i]
            else:
                row['matched_' + str(i)] = 0

        plus3 = 0
        for key in dmatch_count:
            if key >= 3:
                plus3 += dmatch_count[key]
        row['matched_3+'] = plus3

        tp = tp.append(row, ignore_index=True)

    return tp

#get_tp(df, 2)
tp = tp.append(get_tp(df, 20))
print('done')

max attempts
done


In [294]:
#tp.head()

In [295]:
t = tp.copy()
t['TP'] = t.co_count / t.papers
t.sort_values('papers')[-10:]

,akid,co_count,dmatch,id_count,matched_0,matched_1,matched_2,matched_3+,name,papers,TP
297,2.032970e+09,39.0,"{0: 465, 8: 1, 1: 14, 2: 22, 4: 1, 10: 1}",10.0,465.0,14.0,22.0,3.0,j lugtenburg,504.0,0.077381
250,1.998136e+09,8.0,"{0: 513, 1: 8}",10.0,513.0,8.0,0.0,0.0,richard m lerner,521.0,0.015355
252,2.035308e+09,109.0,"{8: 8, 4: 9, 5: 6, 10: 1, 13: 3, 1: 37, 95: 1,...",7.0,496.0,37.0,13.0,59.0,anders hamsten,605.0,0.180165
245,7.332023e+08,526.0,"{3: 49, 0: 90, 7: 59, 4: 51, 2: 94, 1: 119, 8:...",3.0,90.0,119.0,94.0,313.0,robert j soiffer,616.0,0.853896
288,2.004669e+09,14.0,"{0: 606, 1: 14}",10.0,606.0,14.0,0.0,0.0,christopher k ober,620.0,0.022581
153,2.760969e+09,661.0,"{413: 1, 330: 1, 429: 1, 377: 1, 306: 1, 418: ...",10.0,25.0,85.0,435.0,141.0,r frey,686.0,0.963557
157,2.168656e+09,74.0,"{10: 3, 0: 690, 2: 13, 13: 1, 1: 14, 14: 1, 12...",8.0,690.0,14.0,13.0,47.0,pamela s douglas,764.0,0.096859
238,2.026937e+09,964.0,"{61: 3, 54: 11, 271: 5, 55: 14, 58: 4, 60: 4, ...",6.0,5.0,2.0,0.0,962.0,d lelas,969.0,0.994840
335,2.007418e+09,29.0,"{0: 970, 1: 29}",10.0,970.0,29.0,0.0,0.0,anthony atala,999.0,0.029029
48,2.052369e+09,793.0,"{2: 108, 1: 675, 0: 206, 3: 10}",9.0,206.0,675.0,108.0,10.0,alberto briganti,999.0,0.793794


In [296]:
t[t.papers>1].TP.mean()

0.5520682170087895

In [297]:
len(t)

922

# get the probability that will have one or more coauthors - given it is the not the same author P(X|~A)

In [228]:
fp = []

In [233]:

def get_fp(df, n):

    fp = [] # create false positives data frame

    for i in range(n):        
 
        source_id1, name1, id_count1 = get_source_id(df)
        if source_id1 == False: continue
        source_id2, name2, id_count2 = get_source_id(df)
        if source_id2 == False: continue
            
#         source_id1 = 2442043858
#         source_id2 = 1955862764
        
        #print(source_id1, source_id2)
            
        res1 = get_paper_coauthors(source_id1)
        #print(res1)
        if res1 == False: continue    
            
        res2 = get_paper_coauthors(source_id2)
        #print(res2)
        if res2 == False: continue  
            
        paper1 = res1['entities'][random.randint(0, len(res1['entities']))-1]
        paper2 = res2['entities'][random.randint(0, len(res2['entities']))-1]
            
#         print(paper1)
#         print(paper2)
        
        co_id1 = get_coauthor_ids(paper1, source_id1)
        co_id2 = get_coauthor_ids(paper2, source_id2)
        
#         print(co_id1)
#         print(co_id2)
        
        match_count = 0
        for id in co_id1:
            if id in co_id2:
                match_count +=1
        
        fp.append(match_count)

    
    return fp

fp += get_fp(df, 2)
print('done')


done


In [234]:
fp

[0, 0, 0]